In [ ]:
#!nvidia-smi

In [ ]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=0

In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

In [3]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [5]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [6]:
# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_lr = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

# softmax layer (rnn_hidden_size -> num_classes)
lr_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
lr_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_lr, lr_w) + lr_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [8]:
weights = tf.ones([batch_size, sequence_length])
# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [9]:
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.36789 Prediction: yfyyoyoyofofyoy
1 loss: 2.16559 Prediction: yfyyouoyofofyou
2 loss: 1.97769 Prediction: yfyyou youo you
3 loss: 1.80033 Prediction: yf you yono you
4 loss: 1.63274 Prediction: yf you yono you
5 loss: 1.47641 Prediction: yf you yant you
6 loss: 1.33297 Prediction: yf you yant you
7 loss: 1.20346 Prediction: yf you yant you
8 loss: 1.08811 Prediction: yf you yant you
9 loss: 0.986403 Prediction: yf you yant you
10 loss: 0.897253 Prediction: yf you yant you
11 loss: 0.819263 Prediction: yf you yant you
12 loss: 0.750991 Prediction: yf you yant you
13 loss: 0.691149 Prediction: yf you yant you
14 loss: 0.63869 Prediction: yf you yant you
15 loss: 0.592784 Prediction: yf you yant you
16 loss: 0.552759 Prediction: yf you yant you
17 loss: 0.518025 Prediction: yf you yant you
18 loss: 0.488022 Prediction: yf you yant you
19 loss: 0.462211 Prediction: yf you yant you
20 loss: 0.440073 Prediction: yf you yant you
21 loss: 0.421125 Prediction: yf you yant you
22 loss: